## Scenario-I for Gender prediction

In [ ]:
%pip install pandas
%pip install xgboost==0.90
%pip install scikit-learn
%pip install mlxtend
%pip install six
%pip install flask
%pip install boto3


In [18]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
import six
import sys
sys.modules['sklearn.externals.six'] = six
from mlxtend.classifier import StackingCVClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB 
from sklearn import model_selection


# Load data into a pandas DataFrame
data = pd.read_csv("https://myawsdemobucket1995.s3.amazonaws.com/final_capstone_train_test.csv",quoting=3, error_bad_lines=False)


/emr/notebook-env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [19]:
data = data.drop("Unnamed: 0", axis=1)

In [20]:
# Map gender to 0 and 1
gender_map = {'M': 0, 'F': 1}
data['gender'] = data['gender'].map(gender_map)

In [21]:
data_train = data[data['train_test_flag']=='train'] 

In [22]:
data_test = data[data['train_test_flag']=='test'] 

In [23]:
data_train = data_train.drop('train_test_flag', axis=1)
data_test = data_test.drop('train_test_flag', axis=1)

## Model Training Gender

In [24]:
# Split into training and testing sets
X_train = data_train.drop('gender', axis=1)
X_train = X_train.drop('M', axis=1)
X_train = X_train.drop('F', axis=1)

y_train = data_train['gender']

X_test = data_test.drop('gender', axis=1)
X_test = X_test.drop('M', axis=1)
X_test = X_test.drop('F', axis=1)

y_test = data_test['gender']

# Create an XGBoost classifier
model = xgb.XGBClassifier(min_child_weight=1,
                          gamma=0.5,subsample=0.2,
                          max_depth=3,
                          n_estimators=100,
                          learning_rate=0.001)

# Fit the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.708375778155065


In [8]:
import pickle
import boto3
bucket='myawsdemobucket1995'
key='AdCampaignModelGender.pkl'
pickle_byte_obj = pickle.dumps(model)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket,key).put(Body=pickle_byte_obj)

{'ResponseMetadata': {'RequestId': 'VMVDNKTP7NZ1TVAG',
  'HostId': '2Y0nHzJPWdU5wnRfTvK5sNthpkoCssedEtmWu5LYfuSwgHCzYbPGRdoCEsG4/3GU9VVtgSKSYQI=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '2Y0nHzJPWdU5wnRfTvK5sNthpkoCssedEtmWu5LYfuSwgHCzYbPGRdoCEsG4/3GU9VVtgSKSYQI=',
   'x-amz-request-id': 'VMVDNKTP7NZ1TVAG',
   'date': 'Thu, 18 May 2023 15:24:59 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"9316afe6f699b01dcdd4cb844337dec4"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"9316afe6f699b01dcdd4cb844337dec4"',
 'ServerSideEncryption': 'AES256'}

## ## Model Training Age

In [10]:
# Split into training and testing sets
X_train = data_train.drop('age_x', axis=1)
X_train = data_train.drop('gender', axis=1)

y_train = data_train['age_x']

X_test = data_test.drop('age_x', axis=1)
X_test = data_test.drop('gender', axis=1)

y_test = data_test['age_x']

# Create an XGBoost classifier
model = xgb.XGBClassifier(min_child_weight=1,
                          gamma=0.5,subsample=0.2,
                          max_depth=3,
                          n_estimators=100,
                          learning_rate=0.001)

# Fit the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9866440294284098


In [11]:
bucket='myawsdemobucket1995'
key='AdCampaignModelAge.pkl'
pickle_byte_obj = pickle.dumps(model)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket,key).put(Body=pickle_byte_obj)

{'ResponseMetadata': {'RequestId': 'CJT6QE3G7WK6XJ1R',
  'HostId': 'GsM5eGxGCCEXELk6WyqtUoBFxq7FSNHtVUX4tiS+t6YkhFvQ/ma4HgwL0DB04Mkrj7PdbaA9H8s=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'GsM5eGxGCCEXELk6WyqtUoBFxq7FSNHtVUX4tiS+t6YkhFvQ/ma4HgwL0DB04Mkrj7PdbaA9H8s=',
   'x-amz-request-id': 'CJT6QE3G7WK6XJ1R',
   'date': 'Thu, 18 May 2023 15:30:27 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"aa42b01069e3b80cd933c6e3985ffb0d"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"aa42b01069e3b80cd933c6e3985ffb0d"',
 'ServerSideEncryption': 'AES256'}

## Stacked Modal Building

In [25]:
preddata = X_test




In [26]:
# Scaling the data
scaler = preprocessing.StandardScaler()
scaled_df = scaler.fit_transform(X_train)
x_train = pd.DataFrame(scaled_df)
scaled_df = scaler.fit_transform(preddata)
preddata = pd.DataFrame(scaled_df)

In [27]:
# x,y to np 
x_train = x_train.values
y_train = y_train.values

In [28]:
# 1st set of models
clf1 = LogisticRegression()
clf2 = RandomForestClassifier(random_state=1, n_estimators=10) # just for demo have taken 10 trees 
xgb = XGBClassifier()

stacking_demo = StackingCVClassifier(classifiers=[clf1, clf2], meta_classifier=xgb, use_probas=True, cv=3)

In [29]:
# Do CV
for clf, label in zip([clf1, clf2, stacking_demo], 
                      ['lr', 
                       'Random Forest', 
                       'StackingClassifier']):

    scores = model_selection.cross_val_score(clf, x_train, y_train, cv=3, scoring='roc_auc')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.52 (+/- 0.01) [lr]
Accuracy: 0.50 (+/- 0.02) [Random Forest]


/home/emr-notebook/.local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/emr-notebook/.local/lib/python3.7/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/home/emr-notebook/.local/lib/python3.7/site-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "/home/emr-notebook/.local/lib/python3.7/site-packages/mlxtend/classifier/_base_classification.py", line 80, in decision_function
    return self._do_predict(X, self.meta_clf_.decision_function)
AttributeError: 'XGBClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/emr-notebook/

Accuracy: nan (+/- nan) [StackingClassifier]


/home/emr-notebook/.local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/emr-notebook/.local/lib/python3.7/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/home/emr-notebook/.local/lib/python3.7/site-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "/home/emr-notebook/.local/lib/python3.7/site-packages/mlxtend/classifier/_base_classification.py", line 80, in decision_function
    return self._do_predict(X, self.meta_clf_.decision_function)
AttributeError: 'XGBClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/emr-notebook/

### As XGB Classifier is performing better than other algorithms we will be saving the xgb modal as pickle file for the deployment